In [1]:
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
import numpy as np

from qutip import * 
import os

# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
#    'text.usetex': True,              # Enable LaTeX for text rendering
#    'font.family': 'serif',           # Use a serif font family
#    'font.serif': 'Palatino',         # Set Palatino as the serif font
#    'text.latex.preamble': r'\usepackage{amsmath}',
#    'font.size': 20,                   # Font size for general text
#    'axes.titlesize': 20,              # Font size for axis titles
#    'axes.labelsize': 20,              # Font size for axis labels
#    'xtick.labelsize': 20,             # Font size for x-axis tick labels
#    'ytick.labelsize': 20,             # Font size for y-axis tick labels
#    'legend.fontsize': 20,             # Font size for legends
#    'figure.figsize': [8, 6],          # Size of the plot (width x height)
#    'figure.autolayout': True,         # Automatic layout adjustment
#    'savefig.format': 'svg',           # Default format for saving figures
#    'figure.facecolor': 'none',        # Make the figure face color transparent
#    'axes.facecolor': 'none',          # Make the axes face color transparent
#    'savefig.transparent': True        # Save figures with transparent background
})
output_dir = os.getcwd()  # Gets the current working directory
os.makedirs(output_dir, exist_ok=True)

# allows for interactive plots
#%matplotlib notebook

$$
H = H_0 + H_I
$$

$$
H_0 = \hbar \omega_a \sum_{i=1}^{N_{\text{atoms}}} \sigma_+^{(i)} \sigma_-^{(i)} + \hbar \sum_{i,j=1}^{N_\text{{atoms}}} J_{ij} \sigma_+^{(i)} \sigma_-^{(j)} \quad \text{,} \quad J_{ij} = \frac{\alpha}{|r_i - r_j|^3}
$$

$$
H_I = \hbar D \cdot E(t)
\quad \text{,} \quad D = \mu \sum_{i=1}^{N_{\text{atoms}}} \sigma_+^{(i)} + \sigma_-^{(i)}
\quad \text{,} \quad E(t_i) \propto \Omega_{coupling} \cos(\pi (t - t_i)) \delta(t_i)
$$

### Decay operators single case
$$
C_{\text{decay}}^{(i)} = \sqrt{\gamma_0} \sigma_-^{(i)} \quad
C_{\text{dephase}}^{(i)} = \sqrt{\gamma_\phi} \sigma_z^{(i)}
$$

In [2]:
#
# Set the system parameters
#
# ENERGY LANDSCAPE, c = 1, hbar = 1
fixed_lam = 1. #
alpha     = 1. # coupling strength of the dipoles       Fine structure const?

omega_a   = 2 * np.pi / fixed_lam   # energysplitting of the atom, when ground state is set to 0
mu        = 1 * omega_a             # Dipole matrix element of each atom
omega_R   = 1 * omega_a             # Rabi freq coupling to laser field  for first 2 lasers  -> E_field_0 dot dot Dip_op, parallel

# LINBLAD OPS
gamma_0   = .3 # decay rate of the atoms
gamma_phi = .3 # dephasing rate of the atoms

# TOPOLOGY
n_chains = 1 # number of chains
n_rings  = 1 # number of rings
N_atoms  = n_chains * n_rings  # number of atoms

distance = 1. # * fixed_lam # defining topology

# TIME EVOLUTION
last_pulse = 10 # * gamma_0
last_det_t = 1 * last_pulse # last time when the system is measured -> t elem 0...last_det_t - T
time_steps = 120
tau_steps = time_steps
T_steps = time_steps // 10
approx_T_vals = np.linspace(0, last_det_t, T_steps)  # approximate T values

times_t = np.linspace(0, last_det_t, time_steps) # list of times_t
T_indices = np.array([np.abs(times_t - x).argmin() for x in approx_T_vals])

taus = np.linspace(0, last_det_t, time_steps)  # list of times_t for tau
Ts = times_t[T_indices]

mean_pos = [0, 0, 0]  # Mean position (can be any point in 3D space)
sigma_pos = N_atoms / 10  # Standard deviation for position distribution
########################################               define the geometry                 #############################################
def chain_positions(distance, N_atoms):
    Pos = np.zeros((N_atoms, 3))
    for i in range(N_atoms):
        Pos[i, 2] = i * distance
    return Pos
def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0,              0,            1]])
def ring_positions(distance, n_chains):
    Pos = np.zeros((n_chains, 3))
    dphi = 2 * np.pi / n_chains
    if n_chains == 1:
        radius = 0
    else:
        radius = distance / 2 / np.sin(np.pi / n_chains)
    helper = np.array([radius, 0, 0])
    for i in range(n_chains):
        rotation_matrix = z_rotation(dphi * i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos
def cyl_positions(distance, N_atoms, n_chains):
    Pos = np.zeros((N_atoms, 3))
    Pos_chain = chain_positions(distance, N_atoms // n_chains)
    Pos_ring = ring_positions(distance, n_chains)
    for i in range(n_chains):
        Pos[i * (N_atoms // n_chains): (i + 1) * (N_atoms // n_chains)] = Pos_chain + Pos_ring[i]
    return Pos
##################                           help functions                                 ##################
def count_decimal_digits(number):
    # Convert the number to string
    str_number = str(number)

    # Split the string at the decimal point and count the digits after it
    if '.' in str_number:
        return len(str_number.split('.')[1])
    else:
        return 0  # No digits after decimal if it's an integer
tolerance = count_decimal_digits(times_t[1]) - 2
def truncate_number(number, decimals=0):
    factor = 10.0 ** decimals
    return int(number * factor) / factor
def heaviside(x):
    return 1 if x >= 0 else 0
def sample_positions(mean_pos, sigma_pos, N_atoms):
    positions = []
    while len(positions) < N_atoms:
        # Sample a new position for the atom
        new_position = np.random.normal(loc=mean_pos, scale=sigma_pos, size=3)
        # Check if the new position is unique (no duplicates)
        if not any(np.allclose(new_position, pos) for pos in positions):
            positions.append(new_position)
    return np.array(positions)
def sample_frequencies(E0, Delta, N_atoms):
    # Sample N_atoms frequencies from the Gaussian distribution
    frequencies = np.random.normal(loc=E0, scale=Delta/2, size=N_atoms)
    return frequencies
def plot_positive_color_map(x, y, data, T, sum=False, space="real", type="real", positive=False, safe=False):
    """
    Create a color plot of 2D functional data for positive x and y values only.

    Parameters:
        x (np.array): 1D array of x values.
        y (np.array): 1D array of y values.
        data (np.array): 2D array of data values aligned with x and y.
        T (float): Temperature parameter to include in plot title and file name.
        sum (bool): Whether this is a summed plot (True) or not (False).
        space (str): Either 'real' or 'freq' specifying the space of the data.
        type (str): Type of data ('real', 'imag', 'abs', or 'phase'). Used only if space="freq".
        positive (bool): Whether to use only positive values of x and y.
        safe (bool): If True, saves the plot to a file.

    Returns:
        None
    """
    # Convert x and y into 1D arrays if they're not
    x = np.array(x)
    y = np.array(y)

    if positive:
        # Filter for positive x and y values
        positive_x_indices = np.where(x > 0)[0]  # Indices where x > 0
        positive_y_indices = np.where(y > 0)[0]  # Indices where y > 0
        x = x[positive_x_indices]
        y = y[positive_y_indices]
        data = data[np.ix_(positive_y_indices, positive_x_indices)]

    if space == "real":
        colormap = "viridis"
        label = r"$E_{out} \propto P / E_0$"
        title = f"2D Spectrum, Real space at T ={T:.2f} (arb. units)"
        if sum:
            title = f"2D Spectrum, Real space sum (arb. units)"
        x_title = r"$t_{det}$ (arb. units)"
        y_title = r"$\tau_{coh}$ (arb. units)"
    elif space == "freq":
        colormap = "plasma"
        label = r"$E_{out} \propto P / E_0$"
        title = f"2D Spectrum, Freq space at T ={T:.2f} (arb. units)"
        if sum:
            title = f"2D Spectrum, Freq space sum (arb. units)"
        x_title = r"$\omega_{t_{det}}$ (arb. units)"
        y_title = r"$\omega_{\tau_{coh}}$ (arb. units)"
        if type == "real":
            data = np.real(data)
        elif type == "imag":
            data = np.imag(data)
        elif type == "abs":
            data = np.abs(data)
        elif type == "phase":
            data = np.angle(data)
        else:
            raise ValueError("Invalid Type. Must be 'real', 'imag', 'abs', or 'phase'.")
    else:
        raise ValueError("Invalid space. Must be 'real' or 'freq'.")


    # Plot the color map
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(x, y, data, shading="auto", cmap=colormap)
    plt.colorbar(label=label)
    plt.title(title)
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    if safe:
        file_name_combined = (
            f"Classical_lam{fixed_lam:.1f}_alpha={alpha:.2f}_g_0{gamma_0:.2f}_g_phi{gamma_phi:.2f}_"
            f"{n_rings}x{n_chains}_dist={distance:.2f}_positive={positive}_space={space}"
        )
        if space == "freq":
            file_name_combined += f"_type={type}"
        file_name_combined += ".svg"
        save_path_combined = os.path.join(output_dir, file_name_combined)
        plt.savefig(save_path_combined)
    plt.show()
def Hamilton0(distance, n_rings, n_chains):
    N_atoms = n_chains * n_rings
    Pos = cyl_positions(distance, N_atoms, n_chains) # sample_positions(mean_pos, sigma_pos, N_atoms)        #
    atom_frequencies = sample_frequencies(omega_a, 0.0125 * omega_a, N_atoms) # [omega_a] * N_atoms #
    H = 0
    for a in range(N_atoms):
        for b in range(N_atoms):
            op = sm_list[a].dag() * sm_list[b]
            if a != b:
                ra, rb = Pos[a, :], Pos[b, :]
                H += alpha / (np.linalg.norm(rb-ra))**3 * op
            else:
                H += atom_frequencies[a] *op # Diagonals except for |0><0|
    return H
def El_field(t, args):
    t0 = args['time']
    Delta = args['Delta']
    E = np.exp(1j*(args['omega'] * t + args['phi']))
    E += np.conjugate(E)
    # secure the field is 0 outside short range
    E *= np.cos(np.pi * (t - t0) / (2 * Delta))**2 * heaviside(t - (t0 - Delta)) * heaviside(t0 + Delta - t)

    return args['E0'] * E

# Define the ground & the excited states
# atomic dofs
atom_g = basis(N_atoms + 1, 0)
atom_es = [basis(N_atoms + 1, i) for i in range(1, N_atoms + 1)]
# initial state
psiini = atom_g # = |g>_atom

# combined dofs
sm_list = []    # lowering operators of atomic system
Dip_op = 0         # collective sigma_x operator for the system
for i in range(N_atoms):
    op = atom_g * atom_es[i].dag()
    sm_list.append(op)
    Dip_op += mu * op + mu * op.dag()
H0 = Hamilton0(distance, n_rings, n_chains)

# Jump / Expect Operators          # Define the decay collapse and dephasing operator for each spin
# Collapse operators
c_op2 = [np.sqrt(gamma_0) * op for op in sm_list]                         # Individual atom decays
c_op4 = [np.sqrt(gamma_phi) * commutator(op.dag(), op) for op in sm_list] # Individual atom dephasing
c_op_list = c_op2 + c_op4  # Combine all collapse operators
# Expectation operators for measuring populations across atomic ground and excited levels
e_op_list = [ket2dm(basis(N_atoms + 1, i)) for i in range(N_atoms + 1)]

#
# evolution with t
#
# create the time dependant evolution
Omegas = [omega_R, omega_R, omega_R/10] # Probe pulse is smaller
omegas = [omega_a, omega_a, omega_a]    # The laser is on resonant
Delta_ts = [last_pulse/time_steps, last_pulse/time_steps, last_pulse/time_steps] # narrow width of the pulses
HI = [-Dip_op, El_field] # interaction Hamiltonian with function-based time dependence
H = [H0, HI]
options = Options(store_states=True)


#
# PREPROCESSING
#
# Phase cycling
phases = [-1 * i * np.pi / 2 for i in range(4)]

/home/leopold/PycharmProjects/Master_thesis/.venv/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(


In [3]:
#
# CALCULATIONS
#

# (first laser pulse)
times_0 = times_t
# Iterate over the phi values
def compute_first_pulse(phi_1):
    # Set the arguments for this particular phi
    args_0 = {
        'phi': phi_1,
        'time': Delta_ts[0],
        'omega': omegas[0],
        'Delta': Delta_ts[0],
        'E0': Omegas[0]
    }

    # Solve the system using mesolve
    result_0 = mesolve(H, psiini, times_0, c_ops=c_op_list, e_ops=e_op_list, args=args_0, options=options)

    # Prepare the data dictionary for this particular phi value
    current_dict = {}
    for i in range(len(times_0)):
        if (2 * Delta_ts[0] <= times_0[i] <= last_pulse - 2 * (Delta_ts[1] + Delta_ts[2])):
            current_dict[i] = result_0.states[i]
    return current_dict
def compute_second_pulse(phi_2, res_1):
    data_dict_stage2_local = {}
    times_1_dict = {}  # To store times_1 for each combination

    for i, psiini_1 in res_1.items():
        times_1 = times_t[i:]

        args_1 = {
            'phi': phi_2,  # Use phi_2 for the second pulse
            'time': times_1[0] + Delta_ts[1],
            'omega': omegas[1],
            'Delta': Delta_ts[1],
            'E0': Omegas[1]
        }

        # Solve the system using mesolve for the second pulse
        result_1 = mesolve(H, psiini_1, times_1, c_ops=c_op_list, e_ops=e_op_list, args=args_1, options=options)

        current_dict = {}
        for j in range(len(times_1)):  # Save only the states that make sense
            if (times_t[i] + 2 * Delta_ts[1] <= times_1[j] <= last_pulse - 2 * Delta_ts[2]):
                current_dict[j] = result_1.states[j]

        if current_dict:  # Check if the dictionary is not empty
            data_dict_stage2_local[i] = current_dict
            times_1_dict[i] = times_1  # Store times_1 for later access

    return data_dict_stage2_local, times_1_dict
def compute_third_pulse(res_2):
    data_dict = {}
    for i, dic1 in res_2[0].items():
        times_1 = res_2[1][i]
        count = 0
        # Iterate over the T values (waiting times) and calculate the last laser pulse
        for j, psiini_2 in dic1.items():
            times_2 = times_1[j:]
            T_j = times_1[j] + Delta_ts[2] - times_1[0] - Delta_ts[1]  # waiting_time_j
            T_j = truncate_number(T_j, tolerance - 2)
            T_c = truncate_number(Ts[count], tolerance - 2)
            # if the delay T is smaller than the current pulse difference, take next one
            while T_c < T_j and count < len(Ts) - 1:
                count += 1
                T_c = truncate_number(Ts[count], tolerance - 2)

            # ONLY NOW MAKE THE LAST PULSE
            if np.isclose(T_j, T_c):
                # Define the parameters for the last laser pulse
                args_2 = {
                    'phi': 0,  # Last pulse has no phase kick
                    'time': times_2[0] + Delta_ts[2],  # Duration for the 2nd pulse
                    'omega': omegas[2],  # Omega for the 2nd pulse
                    'Delta': Delta_ts[2],  # Delta for the 2nd pulse
                    'E0': Omegas[2]  # E0 for the 2nd pulse
                }

                # Solve the system using mesolve for the last pulse
                result_2 = mesolve(H, psiini_2, times_2, c_ops=c_op_list, e_ops=e_op_list, args=args_2, options=options)

                # Iterate over the states in result_2 and store the ones that meet the condition
                for k in range(len(times_2)):
                    if (times_1[j] + 2 * Delta_ts[2] <= times_2[k] <= last_det_t):
                        tau_i = times_1[0] + Delta_ts[1] - Delta_ts[0]
                        t_k = times_2[k] - times_2[0] - Delta_ts[2]
                        tau_i = truncate_number(tau_i, tolerance - 2)
                        t_k = truncate_number(t_k, tolerance - 2)

                        if T_j not in data_dict:
                            data_dict[T_j] = {}
                        if tau_i not in data_dict[T_j]:
                            data_dict[T_j][tau_i] = {}


                        # Compute the polarization and store it in the dictionary
                        Polarization = expect(Dip_op, result_2.states[k]) / omega_R
                        data_dict[T_j][tau_i][t_k] = Polarization
    return data_dict
# Wrapper function to process each combination of phi_1 and phi_2
def process_phi_combination(phi_1, phi_2):
    # Step 1: Compute the first pulse for phi_1
    first_pulse_data = compute_first_pulse(phi_1)

    # Step 2: Compute the second pulse with the result from the first pulse
    second_pulse_data = compute_second_pulse(phi_2, first_pulse_data)

    # Step 3: Compute the third pulse with the result from the second pulse
    third_pulse_data = compute_third_pulse(second_pulse_data)
    return third_pulse_data

In [4]:
# Initialize results dictionary
all_results = {}
# Parallelize the computation for all combinations of phi_1 and phi_2
max_workers = min(len(phases) ** 2, os.cpu_count())  # Limit workers based on available CPUs
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    # Track each future and corresponding (phi_1, phi_2)
    futures = {executor.submit(process_phi_combination, phi_1, phi_2): (phi_1, phi_2)
               for phi_1 in phases for phi_2 in phases}

    # Collect results as they complete
    for future in futures:
        phi_1, phi_2 = futures[future]  # Retrieve the associated (phi_1, phi_2)
        third_pulse_data = future.result()
        if third_pulse_data is not None:
            all_results[(phi_1, phi_2)] = third_pulse_data

#all_results[(0,0)] # now contains the results keyed by (phi_1, phi_2)

In [5]:
all_results[(0,0)][0.9243697478991].keys()

dict_keys([0.1680672268907, 0.2521008403361, 0.3361344537815, 0.4201680672268, 0.5042016806722, 0.5882352941176, 0.672268907563, 0.7563025210084, 0.8403361344537, 0.9243697478991, 1.0084033613445, 1.0924369747899, 1.1764705882352, 1.2605042016806, 1.344537815126, 1.4285714285714, 1.5126050420168, 1.5966386554621, 1.6806722689075, 1.7647058823529, 1.8487394957983, 1.9327731092436, 2.016806722689, 2.1008403361344, 2.1848739495798, 2.2689075630252, 2.3529411764705, 2.4369747899159, 2.5210084033613, 2.6050420168067, 2.6890756302521, 2.7731092436974, 2.8571428571428, 2.9411764705882, 3.0252100840336, 3.1092436974789, 3.1932773109243, 3.2773109243697, 3.3613445378151, 3.4453781512605, 3.5294117647058, 3.6134453781512, 3.6974789915966, 3.781512605042, 3.8655462184873, 3.9495798319327, 4.0336134453781, 4.1176470588235, 4.2016806722689, 4.2857142857142, 4.3697478991596, 4.453781512605, 4.5378151260504, 4.6218487394957, 4.7058823529411, 4.7899159663865, 4.8739495798319, 4.9579831932773, 5.042016

In [6]:
print(len(list(all_results[(0,0)][0.9243697478991].keys())))
print(len(list(all_results[(0,0)][0.9243697478991][0.1680672268907].keys())))

105
105


In [8]:
# Initialize the averaged_data_dict
averaged_data_dict = {}

# Collect all unique T_j values from all_results dictionary
unique_T_j = {T_key for (phi_1, phi_2) in all_results for T_key in all_results[(phi_1, phi_2)]}

# Iterate over unique T_j values
for T_j in unique_T_j:
    averaged_data_dict[T_j] = {}

    # Collect all unique tau_i values for the current T_j
    unique_tau_i = {tau_key for (phi_1, phi_2) in all_results
                    if T_j in all_results[(phi_1, phi_2)]
                    for tau_key in all_results[(phi_1, phi_2)][T_j]}

    # Iterate over unique tau_i values
    for tau_i in unique_tau_i:
        averaged_data_dict[T_j][tau_i] = {}

        # Collect all unique t_k values for the current (T_j, tau_i)
        unique_t_k = {t_key for (phi_1, phi_2) in all_results
                      if T_j in all_results[(phi_1, phi_2)] and tau_i in all_results[(phi_1, phi_2)][T_j]
                      for t_key in all_results[(phi_1, phi_2)][T_j][tau_i]}

        # Iterate over unique t_k values
        for t_k in unique_t_k:
            # Calculate the sum of polarization values across all phase combinations
            polarization_sum = sum(
                all_results[(phi_1, phi_2)][T_j][tau_i].get(t_k, 0)
                for (phi_1, phi_2) in all_results
                if T_j in all_results[(phi_1, phi_2)] and tau_i in all_results[(phi_1, phi_2)][T_j]
            )

            # Count the number of valid entries for averaging
            valid_count = sum(
                1 for (phi_1, phi_2) in all_results
                if T_j in all_results[(phi_1, phi_2)]
                and tau_i in all_results[(phi_1, phi_2)][T_j]
                and t_k in all_results[(phi_1, phi_2)][T_j][tau_i]
            )

            # Store the averaged polarization value
            averaged_data_dict[T_j][tau_i][t_k] = None#polarization_sum / max(1, valid_count)

#averaged_data_dict[list(unique_T_j)[0]]
print(len(list(averaged_data_dict[0.9243697478991].keys())))
print(len(list(averaged_data_dict[0.9243697478991][0.1680672268907].keys())))
sorted_tau_keys = sorted(averaged_data_dict[0.9243697478991].keys())
print(sorted_tau_keys)

# Extract and sort keys for tau_i = 0.1680672268907 within T_j = 0.9243697478991
sorted_t_keys = sorted(averaged_data_dict[0.9243697478991][0.1680672268907].keys())
sorted_t_keys = np.array(sorted(set(t for tau in taus for t in averaged_data_dict[.9243697478991][tau].keys()))) # TODO SOME PROBLEM HERE
print(sorted_t_keys)
print(len(sorted_t_keys), len(sorted_tau_keys))

105
105
[0.1680672268907, 0.2521008403361, 0.3361344537815, 0.4201680672268, 0.5042016806722, 0.5882352941176, 0.672268907563, 0.7563025210084, 0.8403361344537, 0.9243697478991, 1.0084033613445, 1.0924369747899, 1.1764705882352, 1.2605042016806, 1.344537815126, 1.4285714285714, 1.5126050420168, 1.5966386554621, 1.6806722689075, 1.7647058823529, 1.8487394957983, 1.9327731092436, 2.016806722689, 2.1008403361344, 2.1848739495798, 2.2689075630252, 2.3529411764705, 2.4369747899159, 2.5210084033613, 2.6050420168067, 2.6890756302521, 2.7731092436974, 2.8571428571428, 2.9411764705882, 3.0252100840336, 3.1092436974789, 3.1932773109243, 3.2773109243697, 3.3613445378151, 3.4453781512605, 3.5294117647058, 3.6134453781512, 3.6974789915966, 3.781512605042, 3.8655462184873, 3.9495798319327, 4.0336134453781, 4.1176470588235, 4.2016806722689, 4.2857142857142, 4.3697478991596, 4.453781512605, 4.5378151260504, 4.6218487394957, 4.7058823529411, 4.7899159663865, 4.8739495798319, 4.9579831932773, 5.04201680

KeyError: np.float64(0.0)

In [ ]:
#
# PLOTTING
#
# Function to prepare data for a single T value
def prepare_data_for_T(T):
    taus = np.array(list(averaged_data_dict[T].keys()))
    ts = np.array(sorted(set(t for tau in taus for t in averaged_data_dict[T][tau].keys())))
    print(len(ts), len(taus))
    data = np.zeros((len(ts), len(taus)))  # or dtype=np.complex128 if needed
    # Populate the 2D data array with the corresponding expect_vals
    for i, t in enumerate(ts):  # Iterate over ts
        for j, tau in enumerate(taus):  # Iterate over taus
            if t in averaged_data_dict[T][tau]:  # Check if t exists for the given tau
                data[i, j] = averaged_data_dict[T][tau][t]  # Assign expect_val
    return T, {
        "ts": ts,  # Local ts specific to T
        "taus": taus,  # Local taus specific to T
        "data": data,  # Local data
    }

prepare_data_for_T(.9243697478991)

In [ ]:

# Parallelize the preparation of data for all T values
global_ts, global_taus = None, None
with ProcessPoolExecutor() as executor:
    results = executor.map(prepare_data_for_T, sorted(averaged_data_dict.keys()))
    i = 0
    for T, data in results:
        #print(T)
        if i == 0:
            smallest_key = T
            global_ts = data["ts"]
            print(len(data["ts"]), len(data["taus"]))
            global_taus = data["taus"]
            # Sum all data into global sum
            global_data_time = np.zeros((len(global_ts), len(global_taus)))
            global_data_freq = np.zeros((len(global_ts), len(global_taus)), dtype=np.complex64)

        i += 1
        taus = data["taus"]  # Local taus for the current T
        ts = data["ts"]  # Local ts for the current T
        if len(ts) < 4 or len(taus) < 4:  # Skip small data
            continue

        data_time = data["data"]  # Aligned 2D data

        data_extended_time = np.zeros_like(global_data_time)

        # Map taus and ts to global indices
        for i, t in enumerate(ts):
            for j, tau in enumerate(taus):
                global_i = np.where(global_ts == t)[0]  # Find the index of t in global_ts
                global_j = np.where(global_taus == tau)[0]
                data_extended_time[global_i, global_j] = data_time[i, j]

        data_extended_freq = np.fft.fft2(data_extended_time)
        print(global_ts.shape, global_taus.shape, data_extended_time.shape)
        plot_positive_color_map(global_ts, global_taus, data_extended_time, T, safe = False)
        #plot_positive_color_map(global_t_freqs, global_tau_freqs, data_extended_freq, T, space = "freq", type = "abs", positive = True, safe = False)


        global_data_time += data_extended_time
        global_data_freq += data_extended_freq

global_data_time /= i
global_data_freq /= i

global_t_freqs = (np.fft.fftfreq(len(global_ts), d=(global_ts[1] - global_ts[0])))  # Frequency axis for detection time
global_tau_freqs = (np.fft.fftfreq(len(global_taus), d=(global_taus[1] - global_taus[0])))  # Frequency axis for excitation time

#plot_positive_color_map(global_ts, global_taus, global_data_time, T, sum = True, safe = False)
#plot_positive_color_map(global_t_freqs, global_tau_freqs, global_data_freq, T, sum = True, space = "freq", type = "abs", positive = True, safe = False)